In [1]:
from transformers import TFBertForSequenceClassification
from transformers import FlaxBertForSequenceClassification
from huggingface_hub import HfApi
import shutil
import os

api = HfApi()

models_list = [
    "rubert-tiny2-russian-sentiment"
    # "rubert-base-cased-cedr-russian-emotion",
    # "rubert-tiny2-cedr-russian-emotion",
    # "rubert-base-cased-ru-go-emotions",
    # "rubert-tiny2-ru-go-emotions",
    # "rubert-base-cased-russian-sentiment",
]

for model_name in models_list:

    repo_name = f"seara/{model_name}"

    model = TFBertForSequenceClassification.from_pretrained(repo_name, from_pt=True)
    model.save_pretrained(model_name)
    print("Save TF")

    model = FlaxBertForSequenceClassification.from_pretrained(repo_name, from_pt=True)
    model.save_pretrained(model_name)
    print("Save JAX")

    api.upload_file(
        path_or_fileobj=f"{model_name}/tf_model.h5",
        path_in_repo="tf_model.h5",
        repo_id=repo_name,
    )

    api.upload_file(
        path_or_fileobj=f"{model_name}/flax_model.msgpack",
        path_in_repo="flax_model.msgpack",
        repo_id=repo_name,
    )

    if os.path.exists(model_name):
        shutil.rmtree(model_name)
        print(f"Successfully deleted the folder: {model_name}")
    else:
        print(f"The folder does not exist: {model_name}")

2024-10-29 21:15:43.590573: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1730225743.606804  222635 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1730225743.612609  222635 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-29 21:15:43.633384: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


config.json:   0%|          | 0.00/911 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/117M [00:00<?, ?B/s]

W0000 00:00:1730225770.027651  222635 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model)

Save TF


An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
Some weights of the model checkpoint at seara/rubert-tiny2-russian-sentiment were not used when initializing FlaxBertForSequenceClassification: {('bert', 'embeddings', 'position_ids')}
- This IS expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Save JAX


tf_model.h5:   0%|          | 0.00/117M [00:00<?, ?B/s]

flax_model.msgpack:   0%|          | 0.00/117M [00:00<?, ?B/s]

Successfully deleted the folder: rubert-tiny2-russian-sentiment


In [2]:
from transformers import TFBertForSequenceClassification, AutoTokenizer

model_name = "seara/rubert-base-cased-russian-sentiment"
model = TFBertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "Привет как дела"
inputs = tokenizer(text, return_tensors="tf")
outputs = model(**inputs)
logits = outputs.logits
print(logits)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


tf.Tensor([[ 2.7927408 -2.1833675 -1.0516126]], shape=(1, 3), dtype=float32)


In [4]:
from transformers import FlaxBertForSequenceClassification, AutoTokenizer

model_name = "seara/rubert-base-cased-russian-sentiment"
model = FlaxBertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "Привет как дела"
inputs = tokenizer(text, return_tensors="jax")
outputs = model(**inputs)
logits = outputs.logits
print(logits)

[[ 2.792741  -2.1833653 -1.0516135]]


In [5]:
from transformers import BertForSequenceClassification, AutoTokenizer

model_name = "seara/rubert-base-cased-russian-sentiment"
model = BertForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

text = "Привет как дела"
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits
print(logits)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


tensor([[ 2.7927, -2.1834, -1.0516]], grad_fn=<AddmmBackward0>)


In [2]:
from transformers import pipeline
model = pipeline(model="seara/rubert-base-cased-ru-go-emotions")
model("Привет, ты мне нравишься!")

model.safetensors:   0%|          | 0.00/712M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/395 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.57M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


[{'label': 'love', 'score': 0.5456761121749878}]